# 참고 문서
- (xgboost tutorial) https://www.datacamp.com/community/tutorials/xgboost-in-python
- (gridsearchcv) https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f

In [1]:
from sklearn.datasets import load_boston

boston = load_boston()

In [2]:
print(boston.keys())

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])


In [3]:
boston.data.shape

(506, 13)

In [4]:
boston.feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [7]:
import pandas as pd

df = pd.DataFrame(boston.data)

In [8]:
df.columns = boston.feature_names

In [9]:
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [10]:
df['PRICE'] = boston.target

In [11]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null float64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null float64
TAX        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
PRICE      506 non-null float64
dtypes: float64(14)
memory usage: 55.5 KB


In [13]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


## Build xgboost baseline

In [16]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

x = df.iloc[:,:-1]
y = df.iloc[:,-1]

print(x.shape, y.shape)

(506, 13) (506,)


In [18]:
# optimized data structure for xgboost
dmatrix = xgb.DMatrix(data = x, label = y)

In [19]:
# create train and test sets for cross validation
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size = 0.2, random_state = 123)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(404, 13) (102, 13) (404,) (102,)


In [21]:
# create regressor
xg_reg = xgb.XGBRegressor(
    objective='reg:linear', 
    colsample_bytree=0.3,
    learning_rate=0.1,
    max_depth=5,
    alpha=10, 
    n_estimators=10
)

In [22]:
# train and predict
xg_reg.fit(x_train, y_train)
preds = xg_reg.predict(x_test)

[14:27:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [23]:
# performance check
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 10.397587


## Perform grid search

In [146]:
# https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/xgboost_hyperparameters.html
xgb_params = {
    'objective': 'reg:linear',
}

grid_params = {
    'n_estimators': [50],
    'subsample': [0.9, 1],
    'colsample_byree': [0.2, 0.3, 0.4, 0.5, 0.6],
    'eta': [0.3], # controls the learning rate
    'learning_rate': [0.1],
    'min_child_weight': [1, 2, 3, 4], # min weight for creating a new node. smaller valule might cause overfit
    'max_depth': [2, 3, 4, 5], # deeper trees might causse the model to overfit
    'alpha': [10], # related to L1 norm
    'lambda': [1], # related to L2 norm
    'gamma': [0],
}

In [159]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from xgboost.sklearn import XGBClassifier, XGBRegressor

# https://scikit-learn.org/stable/modules/cross_validation.html#stratified-k-fold

seed = 123
n_fold_inner = 2
skf_inner = KFold(
    n_splits=n_fold_inner, 
    random_state=seed, 
    shuffle=True)

In [162]:
grid = GridSearchCV(
    estimator=XGBRegressor(xgb_params),
    param_grid=grid_params,
    n_jobs=6,
    cv=skf_inner,
    scoring='neg_root_mean_squared_error',
    refit=True,
    verbose=1
)

In [163]:
grid_result = grid.fit(x_train, y_train)

Fitting 2 folds for each of 288 candidates, totalling 576 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done 100 tasks      | elapsed:    0.5s


[15:42:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=6)]: Done 576 out of 576 | elapsed:    1.9s finished


In [164]:
dfGridCvResult = pd.DataFrame(grid.cv_results_)
dfGridCvResult.tail(1)[['mean_test_score', 'std_test_score']]

,mean_test_score,std_test_score
287,-3.822367,0.263154


In [166]:
grid.best_params_

{'alpha': 10,
 'colsample_byree': 0.1,
 'eta': 0.3,
 'gamma': 0,
 'lambda': 1,
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_child_weight': 2,
 'n_estimators': 50,
 'subsample': 0.8}

## Evaluation

In [191]:
from sklearn.metrics import explained_variance_score

pred = grid.best_estimator_.predict(x_test)
print(explained_variance_score(pred ,y_test))

0.8015662450811184


In [197]:
rmse = np.sqrt(mean_squared_error(y_test, pred))
print("RMSE: %f" % (rmse))

RMSE: 3.835178
